In [2]:
from utils import win_loss_utils
import pandas as pd

win_loss_utils.save_data()

In [3]:
all_data, home_data, away_data = win_loss_utils.load_data()
all_data["Start Date"] = pd.to_datetime(all_data["Start Date"], format="%d %b %Y")
home_data["Start Date"] = pd.to_datetime(home_data["Start Date"], format="%d %b %Y")
away_data["Start Date"] = pd.to_datetime(away_data["Start Date"], format="%d %b %Y")

In [4]:
def get_location_counts(data):
    location_data = data.copy()

    location_data_by_ground = location_data.groupby(by=["Ground"])
    location_percents = location_data_by_ground["Result"].value_counts(normalize=True)
    location_raw = location_data_by_ground["Result"].value_counts()

    location_counts = location_data["Ground"].value_counts()
    return location_percents, location_raw, location_counts

def get_location_counts_by_year(data):
    location_data = data.copy()

    location_data["Start Date"] = location_data["Start Date"].apply(lambda date: date.year)

    location_data_by_ground = location_data.groupby(by=["Start Date", "Ground"])
    location_percents = location_data_by_ground["Result"].value_counts(normalize=True)
    location_raw = location_data_by_ground["Result"].value_counts()

    location_counts = location_data["Ground"].value_counts()
    return location_percents, location_raw, location_counts

In [5]:
home_by_location = get_location_counts(home_data)
away_by_location = get_location_counts(away_data)

print(home_by_location)
print(away_by_location)

(Ground     Result
Adelaide   won       0.645161
           draw      0.193548
           lost      0.161290
Brisbane   won       0.800000
           draw      0.200000
Cairns     draw      0.500000
           won       0.500000
Canberra   won       1.000000
Darwin     won       1.000000
Hobart     won       0.666667
           draw      0.166667
           lost      0.166667
Melbourne  won       0.709677
           lost      0.161290
           draw      0.129032
Perth      won       0.655172
           lost      0.206897
           draw      0.137931
Sydney     won       0.593750
           draw      0.281250
           lost      0.125000
Name: Result, dtype: float64, Ground     Result
Adelaide   won       20
           draw       6
           lost       5
Brisbane   won       24
           draw       6
Cairns     draw       1
           won        1
Canberra   won        1
Darwin     won        2
Hobart     won        8
           draw       2
           lost       2
Melbourne  won 

In [58]:
import plotly.express as px

home_game_counts = home_by_location[1]
home_game_counts = home_game_counts.reset_index(name="Matches")
home_game_counts = home_game_counts.rename(columns={"index" : "Ground"})

fig = px.bar(home_game_counts, x="Ground", y="Matches", color="Result")
fig.show()


In [57]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import ground_utils

away_game_counts = get_location_counts_by_year(away_data)[1]
away_game_counts = away_game_counts.reset_index(name="Matches")
away_game_counts = away_game_counts.rename(columns={"index" : "Ground"})

ground_details = ground_utils.load_data()
ground_details = ground_details.set_index("Ground")

away_game_counts = away_game_counts.join(other=ground_details, on="Ground")
away_game_counts = away_game_counts.drop(columns=["Ground Link"])
away_game_counts = away_game_counts.sort_values(by=["Matches"])

away_game_counts_won = away_game_counts.loc[away_game_counts["Result"] == "won"]
away_game_counts_drawn = away_game_counts.loc[away_game_counts["Result"] == "draw"]
away_game_counts_lost = away_game_counts.loc[away_game_counts["Result"] == "lost"]

countries = sorted(list(away_game_counts["Country"].unique()))
fig = make_subplots(rows =  1, cols = len(countries), shared_yaxes=True, subplot_titles=countries)
for i, country in enumerate(countries):
    by_country_won = away_game_counts_won.loc[away_game_counts_won["Country"] == country]
    by_country_draw = away_game_counts_drawn.loc[away_game_counts_drawn["Country"] == country]
    by_country_lost = away_game_counts_lost.loc[away_game_counts_lost["Country"] == country]

    show_legend = True if i == 1 else False

    fig.add_trace(go.Bar(name="Win", x=by_country_won["Ground"], y=by_country_won["Matches"], marker_color='#00CC96', showlegend=show_legend), row=1, col=i + 1)
    fig.add_trace(go.Bar(name="Draw", x=by_country_draw["Ground"], y=by_country_draw["Matches"], marker_color='#636EFA', showlegend=show_legend), row=1, col=i + 1)
    fig.add_trace(go.Bar(name="Loss", x=by_country_lost["Ground"], y=by_country_lost["Matches"], marker_color='#EF553B', showlegend=show_legend), row=1, col=i + 1)

fig.update_layout(height=800, width=2800, xaxis_type='category', barmode='stack')
fig.show()